In [1]:
import pandas as pd

In [2]:
# This is the CSV generated from the mlruns folder
df = pd.read_csv("ablation.csv")

In [3]:
for c in df.columns:
    print(c)

Start Time
Duration
Run ID
Name
Source Type
Source Name
User
Status
batch_size
class_weight
custom_collections
custom_inputRepresentations
custom_noTransposition
custom_npzOutput
custom_outputRepresentations
custom_scrutinizeData
custom_sequenceLength
custom_synthetic
custom_testCollections
custom_testSetOn
custom_texturizeEachTransposition
custom_transpositionKeys
custom_tsvDir
epochs
initial_epoch
max_queue_size
opt_amsgrad
opt_beta_1
opt_beta_2
opt_decay
opt_epsilon
opt_learning_rate
opt_name
sample_weight
shuffle
steps_per_epoch
use_multiprocessing
validation_batch_size
validation_freq
validation_split
validation_steps
workers
Alto35_accuracy
Alto35_loss
Bass35_accuracy
Bass35_loss
HarmonicRhythm7_accuracy
HarmonicRhythm7_loss
LocalKey38_accuracy
LocalKey38_loss
PitchClassSet121_accuracy
PitchClassSet121_loss
RomanNumeral31_accuracy
RomanNumeral31_loss
Soprano35_accuracy
Soprano35_loss
Tenor35_accuracy
Tenor35_loss
TonicizedKey38_accuracy
TonicizedKey38_loss
loss
results_Alto35
res

In [4]:
relevant_columns = ["Name", "val_monitored_accuracy", "val_monitored_loss", "fold"] + [c for c in df.columns if "results" in c]
dfsummary = df[relevant_columns]

In [68]:
dfdict = {"Model": []}
dfdict.update({k: [] for k in dfsummary.columns if "results" in k})
dfdict.update({f"{k}_std": [] for k in dfsummary.columns if "results" in k})
for model in df.Name.unique():
    dfcrossval = dfsummary[dfsummary.Name == model]
    mean = dfcrossval.mean() * 100
    std = dfcrossval.std() * 100
    dfdict["Model"].append(model)
    for c in dfsummary.columns:
        if "results" in c:
            dfdict[c].append(mean[c].round(4))
            dfdict[c + "_std"].append(std[c].round(4))
dfresults = pd.DataFrame(dfdict)
    

/tmp/ipykernel_1868/2935391875.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean = dfcrossval.mean() * 100
/tmp/ipykernel_1868/2935391875.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  std = dfcrossval.std() * 100


In [69]:
dfresults.sort_index(ascending=False)

,Model,results_Alto35,results_Bass35,results_HarmonicRhythm7,results_LocalKey38,results_PitchClassSet121,results_RomanNumeral31,results_Soprano35,results_Tenor35,results_TonicizedKey38,...,results_Alto35_std,results_Bass35_std,results_HarmonicRhythm7_std,results_LocalKey38_std,results_PitchClassSet121_std,results_RomanNumeral31_std,results_Soprano35_std,results_Tenor35_std,results_TonicizedKey38_std,results_satbRomanNumeral_std
10,baseline,68.06,74.52,78.94,78.78,73.46,61.26,72.26,71.30,80.00,...,2.2109,2.1673,0.9450,3.3745,1.9295,2.7718,1.6502,2.5040,1.8193,3.1382
9,input_35_one-hot_spelling,68.08,74.36,79.18,78.66,73.52,61.38,72.40,71.22,80.14,...,2.1902,2.3093,1.4584,3.2020,1.8647,2.4692,1.8668,2.2264,1.8849,3.1746
8,input_no_onset,66.66,72.50,57.72,79.36,72.34,61.20,71.46,69.62,80.70,...,1.7024,1.8655,2.0801,3.6712,1.6757,2.3854,1.7587,1.8485,1.9786,2.9995
7,input_no_bottom,54.98,58.02,78.96,79.10,73.72,61.70,67.24,55.94,80.30,...,2.2454,2.3285,1.4258,3.0232,1.8780,2.7946,1.7869,2.5185,1.8152,2.5948
6,input_no_chroma,56.98,72.84,74.26,72.42,51.30,43.08,55.52,60.98,70.02,...,1.9512,1.9100,1.1014,4.1692,2.3654,2.1673,1.9804,2.3690,3.1618,2.3452
5,conv_single_block,67.80,74.08,79.00,78.36,73.52,61.06,72.22,71.12,80.02,...,2.0893,2.0849,1.4983,3.2470,1.8966,2.7610,2.0067,2.2906,1.9753,3.2604
4,conv_constant_filters,67.74,74.30,78.90,78.32,73.02,60.26,71.70,71.08,79.74,...,2.1961,2.5090,1.4036,3.2965,2.2016,2.7907,2.1610,2.4304,2.0538,3.4535
3,conv_noconv,67.68,74.20,78.66,78.90,73.46,60.82,72.12,71.16,80.76,...,1.9305,1.9761,1.8849,3.7242,1.4206,2.7824,1.6362,1.9463,1.9565,3.0452
2,dense_lineardense,68.26,74.98,79.32,78.30,74.26,61.58,72.96,71.60,80.08,...,2.0983,2.2443,1.6947,3.3324,1.8105,2.6052,1.8393,2.0628,1.9512,3.0583
1,gru_nogru,62.52,69.56,71.98,60.30,67.46,50.34,67.24,65.06,67.64,...,1.4772,1.8284,1.2317,2.9757,1.4363,2.5472,1.4843,1.5662,2.3829,1.8188


In [70]:
dfbaseline = dfresults[dfresults.Model == "baseline"]
for model in dfresults.Model.unique():
    if model == "baseline":
        continue
    dfmodel = dfresults[dfresults.Model == model]
    for metric in dfmodel.columns:
        if not metric.startswith("results"):
            continue
        # print(metric)
        # print(dfmodel[metric], dfbaseline[metric])
        dfresults.loc[dfmodel.index, metric] = dfmodel[metric].to_numpy() - dfbaseline[metric].to_numpy() 

In [74]:
dfresults.round(1).sort_index(ascending=False)

,Model,results_Alto35,results_Bass35,results_HarmonicRhythm7,results_LocalKey38,results_PitchClassSet121,results_RomanNumeral31,results_Soprano35,results_Tenor35,results_TonicizedKey38,...,results_Alto35_std,results_Bass35_std,results_HarmonicRhythm7_std,results_LocalKey38_std,results_PitchClassSet121_std,results_RomanNumeral31_std,results_Soprano35_std,results_Tenor35_std,results_TonicizedKey38_std,results_satbRomanNumeral_std
10,baseline,68.1,74.5,78.9,78.8,73.5,61.3,72.3,71.3,80.0,...,2.2,2.2,0.9,3.4,1.9,2.8,1.7,2.5,1.8,3.1
9,input_35_one-hot_spelling,0.0,-0.2,0.2,-0.1,0.1,0.1,0.1,-0.1,0.1,...,-0.0,0.1,0.5,-0.2,-0.1,-0.3,0.2,-0.3,0.1,0.0
8,input_no_onset,-1.4,-2.0,-21.2,0.6,-1.1,-0.1,-0.8,-1.7,0.7,...,-0.5,-0.3,1.1,0.3,-0.3,-0.4,0.1,-0.7,0.2,-0.1
7,input_no_bottom,-13.1,-16.5,0.0,0.3,0.3,0.4,-5.0,-15.4,0.3,...,0.0,0.2,0.5,-0.4,-0.1,0.0,0.1,0.0,-0.0,-0.5
6,input_no_chroma,-11.1,-1.7,-4.7,-6.4,-22.2,-18.2,-16.7,-10.3,-10.0,...,-0.3,-0.3,0.2,0.8,0.4,-0.6,0.3,-0.1,1.3,-0.8
5,conv_single_block,-0.3,-0.4,0.1,-0.4,0.1,-0.2,-0.0,-0.2,0.0,...,-0.1,-0.1,0.6,-0.1,-0.0,-0.0,0.4,-0.2,0.2,0.1
4,conv_constant_filters,-0.3,-0.2,-0.0,-0.5,-0.4,-1.0,-0.6,-0.2,-0.3,...,-0.0,0.3,0.5,-0.1,0.3,0.0,0.5,-0.1,0.2,0.3
3,conv_noconv,-0.4,-0.3,-0.3,0.1,0.0,-0.4,-0.1,-0.1,0.8,...,-0.3,-0.2,0.9,0.3,-0.5,0.0,-0.0,-0.6,0.1,-0.1
2,dense_lineardense,0.2,0.5,0.4,-0.5,0.8,0.3,0.7,0.3,0.1,...,-0.1,0.1,0.7,-0.0,-0.1,-0.2,0.2,-0.4,0.1,-0.1
1,gru_nogru,-5.5,-5.0,-7.0,-18.5,-6.0,-10.9,-5.0,-6.2,-12.4,...,-0.7,-0.3,0.3,-0.4,-0.5,-0.2,-0.2,-0.9,0.6,-1.3


In [79]:
dfresults.sort_index(ascending=False).round(1).to_csv("results.tsv", sep="\t")